# Max Flow Applications

The purpose of this assignment is to investigate applications of finding a Max Flow. The problem asks you to design and implement an algorithm for shipping a material between nodes with different supply and demand requirements.

* Please write code *only* in the bodies of the two functions, that is, following the TODO comments.
* Be careful not to use varibles defined outside of the functions.
* Breaking the two above rules may lead to 0 grades.

## Movie distribution

First solve Problem 2 from hw3-t. 

Now suppose a movie distributor would like to ship a copy of a film from CA to every other state. There are therefore 48 units to ship out of CA, and each other state receives 1 unit. 

The dataset contiguous-usa.dat lists the adjacent states in the US. Each line lists two adjacent states; thus AK and HI are omitted, but DC is included in the data. The following code reads in the graph of US states.

In [23]:
import networkx as nx
G = nx.Graph()

usa = open('contiguous-usa.dat')
for line in usa:
    s1, s2 = line.strip().split()
    G.add_edge(s1, s2)

We now encode the demands into the graph.

In [24]:
for state in G.nodes():
    if state != 'CA':
        G.node[state]['demand'] = 1
G.node['CA']['demand'] = -48

We will assign a uniform capacity of 16 to each edge. Since CA has only three adjacent states, this is the smallest possible uniform capacity that allows one to ship all 48 units out of CA. As we have created an undirected graph, and flows have directions, we first convert the graph to a directed graph.

In [25]:
G = nx.DiGraph(G)
uniform_capacity = 16
for (s1, s2) in G.edges():
    G.edge[s1][s2]['capacity'] = uniform_capacity

Complete the following function to implement your algorithm to find a flow with demands. Your function should work correctly for any input, not just the movie instance considered here. As always, you are encouraged to define auxiliary functions as needed for clarity.

In [26]:
def flow_with_demands(graph):
    """Computes a flow with demands over the given graph.
    
    Args:
        graph: A directed graph with nodes annotated with 'demand' properties and edges annotated with 'capacity' 
            properties.
        
    Returns:
        A dict of dicts containing the flow on each edge. For instance, flow[s1][s2] should provide the flow along
        edge (s1, s2).
        
    Raises:
        NetworkXUnfeasible: An error is thrown if there is no flow satisfying the demands.
    """
    # TODO: Implement the function.
    
    def draw_residual(G, f):
        """Computes the residual graph given graph and flow.

        Args:
            graph G: A directed graph with 'capacity' 
            f: the flow

        Returns:
            Gf: The corresponding residual graph

        """
        Gf = nx.DiGraph()
        Gf.add_nodes_from(nx.nodes(G))
        for (s1, s2) in nx.edges(G):
            if f[s1][s2] < G.edge[s1][s2]['capacity']:
                Gf.add_edge(s1, s2, {'capacity': G.edge[s1][s2]['capacity'] - f[s1][s2]})
            if f[s1][s2] > 0:
                Gf.add_edge(s2, s1, {'capacity': f[s1][s2]})
        return Gf
    
    
    def bfs(G, s, t):
        """Find an s-t path in graph G

        Args:
            graph G: A directed graph with 'capacity' 
            s: the source
            t: the sink

        Returns:
            path: a list of edges of s-t path, None if not exists
            path_capacity: the bottleneck capacity of the s-t path, None if not exists
            flag: True if there exists an s-t path, else False.

        """
        queue = [(s, [s])]

        while queue:
            (vertex, path) = queue.pop(0)
            for next_node in set(G.neighbors(vertex)) - set(path):
                if next_node == t:
                    long_path = path + [next_node]
                    # try to compute output
                    st_path = list()
                    path_capacity = float('inf')
                    for temp_i in range(len(long_path) - 1):
                        st_path.append((long_path[temp_i], long_path[temp_i + 1]))
                        path_capacity = min(path_capacity, G.edge[long_path[temp_i]][long_path[temp_i + 1]]['capacity'])
                    return st_path, path_capacity, True

                else:
                    queue.append((next_node, path + [next_node]))

        return None, None, False
    
    
    def max_flow(G, s, t):
        """Computes the max flow over the given graph and source and sink.

        Args:
            graph: A directed graph with nodes annotated with 'demand' properties and edges annotated with 'capacity' 
                properties.
            s: the source
            t: the sink

        Returns:
            flow_value : integer, float
            Value of the maximum flow.

            flow_dict : dict
            A dictionary containing the value of the flow that goes through each edge.

        """
        # initialiing the flow
        flow_dict_l = dict()
        for (s1, s2) in nx.edges(G):
            flow_dict_l.setdefault(s1, {})[s2] = 0
        flow_dict_l["t'"] = {}

        # construct the residual graph Gf
        Gf = draw_residual(G, flow_dict_l)

        # define the path storing the s-t path
        path, path_capacity, flag = bfs(Gf, s, t)
        
        if flag == False:
            raise nx.NetworkXUnfeasible("There is no feasible flow satisfying the demands.")
            
        while flag: # if there exists an s-t path in Gf
            for (path_s, path_e) in path:
                if (path_s, path_e) in nx.edges(G):
                    flow_dict_l[path_s][path_e] += path_capacity
                else:
                    flow_dict_l[path_e][path_s] -= path_capacity
            Gf = draw_residual(G, flow_dict_l)
            path, path_capacity, flag = bfs(Gf, s, t)

        # calculate the value of the flow
        flow_value = 0
        for source_out, capacity in flow_dict_l[s].iteritems():
            flow_value += capacity

        return flow_value, flow_dict_l

    
    # construct a new graph G1, and run FF algo to find the max flow in G1
    G1 = nx.DiGraph()
    G1.add_nodes_from(nx.nodes(G))
    for (s1, s2) in nx.edges(G):
        G1.add_edge(s1, s2, {'capacity': G.edge[s1][s2]['capacity']})
    # add nodes s' and t'
    G1.add_node("s'")
    G1.add_node("t'")
    sink_demand = 0
    # add edges s'- x and y - t', x is a source and y is a sink
    for state in nx.nodes(G):
        if G.node[state]['demand'] < 0: # state is a source
            G1.add_edge("s'", state, {'capacity': -G.node[state]['demand']})
        elif G.node[state]['demand'] > 0: # state is a sink
            sink_demand += G.node[state]['demand']
            G1.add_edge(state, "t'", {'capacity': G.node[state]['demand']})
    # run FF algo in the new graph G1
    flow_value, flow_dict = max_flow(G1, "s'", "t'")

    # calculate the flow_dict in the original graph G
    flow_dict.pop("s'", 0)
    flow_dict.pop("t'", 0)
    for origin_node, corr_flow in flow_dict.iteritems():
        corr_flow.pop("t'", 0)
    
    # raise error if there is no flow satisfying the demands
    if flow_value == sink_demand:
        return flow_dict
    else:
        raise nx.NetworkXUnfeasible("There is no feasible flow satisfying the demands.")
        
        

To verify that your solution is correct, implement a function that computes the total flow into each node (which will be negative for supply nodes).

In [27]:
def divergence(flow):
    """Computes the total flow into each node according to the given flow dict.
    
    Args:
        flow: the flow dict recording flow between nodes.
        
    Returns:
        A dict of the net flow into each node.
    """
    # TODO: Implement the function.
    net_flow = dict.fromkeys(nx.nodes(G), 0) # initialize the dict
    for each_start, multi_flow in flow.iteritems():
        for each_end, each_flow in multi_flow.iteritems():
            net_flow[each_start] -= each_flow
            net_flow[each_end] += each_flow
    return net_flow

The following code performs a sanity check on your function (but does not completely confirm correctness).

In [28]:
flow = flow_with_demands(G)
div = divergence(flow)
print "Flow satisfies all demands:", all(div[n] == G.node[n]['demand'] for n in G.nodes())

Flow satisfies all demands: True
